# Introduction
This notebook demonstrates a bidirectional integration between the LAMMPS molecular dynamics engine and a real-time visualization pipeline based on ANARI, using Ascent for in situ data streaming. The goal is to enable interactive exploration and control of scientific simulations by coupling numerical state with a rendering backend. The pipeline supports real-time feedback, allowing users to inspect and manipulate simulation parameters during runtime.

![DAIMSL Project](media/DAIMSL_4.png)

![HPC Integration](media/DAIMSL_interface_0.png)

## Set Environment

In [10]:
import os
import copy

# Store a deep copy of the original environment
original_env = copy.deepcopy(os.environ)


In [30]:
# Reset os.environ to its original state
os.environ.clear()
os.environ.update(original_env)

In [11]:
from dotenv import dotenv_values
import os

def append_env_from_files(*env_files):
    """
    Load and append environment variables from one or more .env files.
    If a variable already exists, prepend the new value unless it's already included.
    """
    for file in env_files:
        env_vars = dotenv_values(file)
        for key, new_val in env_vars.items():
            current_val = os.environ.get(key, "")
            if current_val:
                if new_val not in current_val.split(":"):
                    os.environ[key] = f"{new_val}:{current_val}"
            else:
                os.environ[key] = new_val

# LAMMPS
LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) is a classical molecular dynamics code developed for parallel simulations of particles interacting under various force fields. It supports a wide range of models for solid-state materials, soft matter, and coarse-grained systems, and can be extended via user-defined fixes and external control interfaces.

## Load Variables

In [12]:
append_env_from_files("python_vars/mpich_vars")
append_env_from_files("python_vars/zlib_vars")
append_env_from_files("python_vars/hdf5_vars")
append_env_from_files("python_vars/lammps_vars")

## Example: Cube

In [ ]:
using namespace LAMMPS_NS;

int main(int argc, char **argv)
{
  MPI_Init(&argc, &argv);  
  
  // Create LAMMPS instance
  LAMMPS *lmp = new LAMMPS(argc, argv, MPI_COMM_WORLD);

  // Run input script (e.g., in.lj)
  lmp->input->file();

  // Dump mpg
  lmp->input->one("dump mv all movie 1 lammps_box.mpg type type");
  lmp->input->one("dump_modify mv pad 4");

  for(int i=0; i<20; ++i) {
    lmp->input->one("run 10 post no");
  }
  
  // Cleanup
  delete lmp;

  MPI_Finalize(); 
  return 0;
}

## Compile and Run

In [13]:
!examples/lammps/cube/build_lammps.sh

In [14]:
!examples/lammps/cube/run.sh &> /dev/null

## Visualize LAMMPS Output

In [6]:
!ffmpeg -y -i lammps_box.mpg -c:v libx264 lammps_box.mp4 &> /dev/null

In [17]:
from IPython.display import Video
Video("media/lammps_box.mp4")

# Ascent
Ascent is an in situ visualization and analysis library designed for high-performance computing workloads. It uses the Conduit data model to receive simulation state and supports multiple rendering backends through a high-level action interface. 

Ascent is used in this workflow to extract, convert, and transmit LAMMPS simulation data to downstream visualization components without disrupting the simulation loop.

## Load Variables

In [7]:
append_env_from_files("python_vars/ascent_vars")
append_env_from_files("python_vars/conduit_vars")

## Example: Cube

In [ ]:
void lammps_atoms_to_conduit_mesh(LAMMPS *lmp, Node &mesh)
{
  Atom *atom = lmp->atom;
  int nlocal = atom->nlocal;
  double **x = atom->x;
  double **v = atom->v;  // Velocity array

  // Allocate temporary arrays
  float *x_vals = new float[nlocal];
  float *y_vals = new float[nlocal];
  float *z_vals = new float[nlocal];

  float *vx_vals = new float[nlocal];
  float *vy_vals = new float[nlocal];
  float *vz_vals = new float[nlocal];

  for (int i = 0; i < nlocal; i++) {
    // Position
    x_vals[i] = static_cast<float>(x[i][0]);
    y_vals[i] = static_cast<float>(x[i][1]);
    z_vals[i] = static_cast<float>(x[i][2]);
    // Velocity
    vx_vals[i] = static_cast<float>(v[i][0]);
    vy_vals[i] = static_cast<float>(v[i][1]);
    vz_vals[i] = static_cast<float>(v[i][2]);
  }

  float *speed_vals = new float[nlocal];
  for (int i = 0; i < nlocal; ++i) {
      float vx = vx_vals[i];
      float vy = vy_vals[i];
      float vz = vz_vals[i];
      speed_vals[i] = std::sqrt(vx * vx + vy * vy + vz * vz);
  }
  // Create explicit coordinate set
  mesh["coordsets/coords/type"] = "explicit";
  mesh["coordsets/coords/values/x"].set_external(x_vals, nlocal);
  mesh["coordsets/coords/values/y"].set_external(y_vals, nlocal);
  mesh["coordsets/coords/values/z"].set_external(z_vals, nlocal);

  // Define topology (points)
  mesh["topologies/topo/type"] = "points";
  mesh["topologies/topo/coordset"] = "coords";

  // Add velocity as a vector field
  mesh["fields/velocity/association"] = "vertex";
  mesh["fields/velocity/topology"] = "topo";
  mesh["fields/velocity/values"].set_external(speed_vals, nlocal);

  // Add simulation time
  mesh["state/time"] = lmp->update->ntimestep;
}

In [ ]:

int main(int argc, char **argv)
{
  MPI_Init(&argc, &argv);

  // Get Ascent about info
  Node ascent_info;
  ascent::about(ascent_info);
  std::cout << "Ascent Config:\n" << ascent_info.to_yaml() << std::endl;
  
  // Create LAMMPS instance
  LAMMPS *lmp = new LAMMPS(argc, argv, MPI_COMM_WORLD);

  // Run input script (e.g., in.lj)
  lmp->input->file();

  for(int i=0; i<20; ++i) {
    lmp->input->one("run 10 post no");
    
    // Convert atom data to Conduit mesh
    Node mesh;
    lammps_atoms_to_conduit_mesh(lmp, mesh);
    
    // Start Ascent and publish mesh
    Ascent a;
    //   Node ascent_opts;
    // ascent_opts["mpi_comm"] = MPI_Comm_c2f(MPI_COMM_WORLD);
    // a.open(ascent_opts);
    a.open();
    a.publish(mesh);
    
    // Define actions
    Node actions;
    Node &add_act = actions.append();
    add_act["action"] = "add_scenes";
    
    Node &scenes = add_act["scenes"];
    scenes["s1/plots/p1/type"] = "pseudocolor";
    scenes["s1/plots/p1/field"] = "velocity";
    scenes["s1/image_prefix"] = "lammps_cube_";
    scenes["s1/image_number_format"] = "%04d";
    scenes["s1/image_dir"] = "output_images";
    
    std::cout << "Ascent Actions:\n" << actions.to_yaml() << std::endl;
    
    // Render
    a.execute(actions);

    a.close();
  }
  
  // Cleanup
  delete lmp;

  MPI_Finalize(); 
  return 0;
}

In [8]:
!examples/lammps/cube/build_lammps_ascent.sh

In [9]:
!examples/lammps/cube/run.sh &> /dev/null

![Cube Rendered using Ascent - PNGs](examples/lammps/cube/output_images/cube_render_000000.png)

In [ ]:
!examples/lammps/cube/genmp4.sh

In [18]:
from IPython.display import Video
Video("media/lammps_box_ascent.mp4")

# ANARI
ANARI (Analytic Rendering Interface) is a rendering API that provide type-safe wrappers that abstracts scene description and rendering tasks across heterogeneous devices. It enables portable rendering by decoupling application logic from graphics backends and supports in-memory hierarchical object trees to describe full frame content, including surface geometry and volumetric data.

ANARI provides semantics for rendering engines to implement extensions, support asynchronous scene updates, and access zero-copy memory, facilitating interactive and low-latency visualization. In this integration, ANARI is used to render particle and field data exported from LAMMPS via Ascent.

## Load Variables

In [76]:
# Reset os.environ to its original state
os.environ.clear()
os.environ.update(original_env)

In [77]:
!echo $PATH

/home/droaperdomo/python_venv/venv/bin:/home/droaperdomo/.vscode-server/bin/c306e94f98122556ca081f527b466015e1bc37b0/bin/remote-cli:/home/droaperdomo/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Python312/Scripts/:/mnt/c/Python312/:/mnt/c/Program Files/Oculus/Support/oculus-runtime:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files (x86)/dotnet/:/mnt/c/ProgramData/chocolatey/bin:/mnt/c/Program Files/nodejs/:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem:/mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/:/mnt/c/WINDOWS/System32/OpenSSH/:/mnt/c/Program Files/NVIDIA Corporation/NVIDIA App/NvDLISR:/mnt/c/Program Files (x86)/Windows Kits/10/Windows Performance Toolkit/:/mnt/c/Users/droaperdom

In [78]:
append_env_from_files("python_vars/mpich_vars")
append_env_from_files("python_vars/zlib_vars")
append_env_from_files("python_vars/hdf5_vars")
append_env_from_files("python_vars/lammps_vars")

In [79]:
append_env_from_files("python_vars/ascent_anari_vars")
append_env_from_files("python_vars/conduit_anari_vars")

## Example: Cube

In [ ]:
void lammps_atoms_to_conduit_mesh(LAMMPS *lmp, Node &mesh)
{
  Atom *atom = lmp->atom;
  int nlocal = atom->nlocal;
  double **x = atom->x;
  double **v = atom->v;  // Velocity array

  // Allocate temporary arrays
  float *x_vals = new float[nlocal];
  float *y_vals = new float[nlocal];
  float *z_vals = new float[nlocal];

  float *vx_vals = new float[nlocal];
  float *vy_vals = new float[nlocal];
  float *vz_vals = new float[nlocal];

  for (int i = 0; i < nlocal; i++) {
    // Position
    x_vals[i] = static_cast<float>(x[i][0]);
    y_vals[i] = static_cast<float>(x[i][1]);
    z_vals[i] = static_cast<float>(x[i][2]);
    // Velocity
    vx_vals[i] = static_cast<float>(v[i][0]);
    vy_vals[i] = static_cast<float>(v[i][1]);
    vz_vals[i] = static_cast<float>(v[i][2]);
  }

  float *speed_vals = new float[nlocal];
  for (int i = 0; i < nlocal; ++i) {
      float vx = vx_vals[i];
      float vy = vy_vals[i];
      float vz = vz_vals[i];
      speed_vals[i] = std::sqrt(vx * vx + vy * vy + vz * vz);
  }
  // Create explicit coordinate set
  mesh["coordsets/coords/type"] = "explicit";
  mesh["coordsets/coords/values/x"].set_external(x_vals, nlocal);
  mesh["coordsets/coords/values/y"].set_external(y_vals, nlocal);
  mesh["coordsets/coords/values/z"].set_external(z_vals, nlocal);

  // Define topology (points)
  mesh["topologies/topo/type"] = "points";
  mesh["topologies/topo/coordset"] = "coords";

  // Add velocity as a vector field
  mesh["fields/velocity/association"] = "vertex";
  mesh["fields/velocity/topology"] = "topo";
  mesh["fields/velocity/values"].set_external(speed_vals, nlocal);

  // Add simulation time
  mesh["state/time"] = lmp->update->ntimestep;
}

In [ ]:
int main(int argc, char **argv)
{
  MPI_Init(&argc, &argv);

  // Get Ascent about info
  Node ascent_info;
  ascent::about(ascent_info);
  std::cout << "Ascent Config:\n" << ascent_info.to_yaml() << std::endl;
  
  // Create LAMMPS instance
  LAMMPS *lmp = new LAMMPS(argc, argv, MPI_COMM_WORLD);

  // Run input script (e.g., in.lj)
  lmp->input->file();

  for(int i=0; i<2; ++i) {
    lmp->input->one("run 10 post no");
    
    // Convert atom data to Conduit mesh
    Node mesh;
    lammps_atoms_to_conduit_mesh(lmp, mesh);
    
    // Start Ascent and publish mesh
    Ascent a;
    a.open();
    a.publish(mesh);
    
    // Define actions
    Node actions;
    Node &add_act = actions.append();
    
    std::cout << "Ascent Actions:\n" << actions.to_yaml() << std::endl;
    
    // Render
    a.execute(actions);

    a.close();
  }
  
  // Cleanup
  delete lmp;

  MPI_Finalize(); 
  return 0;
}

In [ ]:
- action: add_scenes
  scenes:
    scene1:
      plots:
        plot1:
          type: pseudocolor
          field: velocity
          points:  
            radius: 0.3  # Increased particle radius
      renders:
        r1:
          image_prefix: "cube_render"
          camera:
            look_at: [0.0, 0.0, 0.0]
            position: [25.0, 25.0, 25.0]
          image_width: 1024
          image_height: 768

![Cube Rendered using Anari](examples/lammps/cube/output_images/anari_render_000000.png)

In [80]:
!examples/lammps/cube/build_lammps_ascent_anari.sh

In [ ]:
!examples/lammps/cube/run_ascent_anari.sh

In [ ]:
!examples/lammps/cube/genmp4.sh

In [ ]:
from IPython.display import Video
Video("lammps_box_anari.mp4")

# ANARI-USD
ANARI-USD is an implementation of an ANARI rendering device that outputs data in the USD (Universal Scene Description) format. It allows ANARI applications to render USD scene graphs directly or engage with rendering pipelines to visualize ANARI content using USD-compatible tools and frameworks, including NVIDIA’s Omniverse.

This backend is used to stream LAMMPS state into a USD scene for rendering and visualization.


## Load Variables

In [ ]:
# Reset os.environ to its original state
os.environ.clear()
os.environ.update(original_env)

In [ ]:
!echo $PATH

/home/droaperdomo/ascent_anari/obj_bin/install/conduit-v0.9.4/bin:/home/droaperdomo/ascent_anari/obj_bin/install/ascent-checkout/bin:/home/droaperdomo/lammps/obj_bin:/home/droaperdomo/tools/hdf5_bin/bin:/home/droaperdomo/tools/zlib_bin/bin:/home/droaperdomo/tools/mpich/bin:/home/droaperdomo/python_venv/venv/bin:/home/droaperdomo/.vscode-server/bin/c306e94f98122556ca081f527b466015e1bc37b0/bin/remote-cli:/home/droaperdomo/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Python312/Scripts/:/mnt/c/Python312/:/mnt/c/Program Files/Oculus/Support/oculus-runtime:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files (x86)/dotnet/:/mnt/c/ProgramData/chocolatey/bin:/mnt/c/Program Files/nodejs/:/mnt/c/WINDOWS/system32:/mnt/

In [ ]:
append_env_from_files("python_vars/mpich_vars")
append_env_from_files("python_vars/zlib_vars")
append_env_from_files("python_vars/hdf5_vars")
append_env_from_files("python_vars/lammps_vars")

In [ ]:
append_env_from_files("python_vars/ascent_anari_vars")
append_env_from_files("python_vars/conduit_anari_vars")

## Example: Cube

In [ ]:
void lammps_atoms_to_conduit_mesh(LAMMPS *lmp, Node &mesh)
{
  Atom *atom = lmp->atom;
  int nlocal = atom->nlocal;
  double **x = atom->x;
  double **v = atom->v;  // Velocity array

  // Allocate temporary arrays
  float *x_vals = new float[nlocal];
  float *y_vals = new float[nlocal];
  float *z_vals = new float[nlocal];

  float *vx_vals = new float[nlocal];
  float *vy_vals = new float[nlocal];
  float *vz_vals = new float[nlocal];

  for (int i = 0; i < nlocal; i++) {
    // Position
    x_vals[i] = static_cast<float>(x[i][0]);
    y_vals[i] = static_cast<float>(x[i][1]);
    z_vals[i] = static_cast<float>(x[i][2]);
    // Velocity
    vx_vals[i] = static_cast<float>(v[i][0]);
    vy_vals[i] = static_cast<float>(v[i][1]);
    vz_vals[i] = static_cast<float>(v[i][2]);
  }

  float *speed_vals = new float[nlocal];
  for (int i = 0; i < nlocal; ++i) {
      float vx = vx_vals[i];
      float vy = vy_vals[i];
      float vz = vz_vals[i];
      speed_vals[i] = std::sqrt(vx * vx + vy * vy + vz * vz);
  }
  // Create explicit coordinate set
  mesh["coordsets/coords/type"] = "explicit";
  mesh["coordsets/coords/values/x"].set_external(x_vals, nlocal);
  mesh["coordsets/coords/values/y"].set_external(y_vals, nlocal);
  mesh["coordsets/coords/values/z"].set_external(z_vals, nlocal);

  // Define topology (points)
  mesh["topologies/topo/type"] = "points";
  mesh["topologies/topo/coordset"] = "coords";

  // Add velocity as a vector field
  mesh["fields/velocity/association"] = "vertex";
  mesh["fields/velocity/topology"] = "topo";
  mesh["fields/velocity/values"].set_external(speed_vals, nlocal);

  // Add simulation time
  mesh["state/time"] = lmp->update->ntimestep;
}

In [ ]:
int main(int argc, char **argv)
{
  MPI_Init(&argc, &argv);

  // Get Ascent about info
  Node ascent_info;
  ascent::about(ascent_info);
  std::cout << "Ascent Config:\n" << ascent_info.to_yaml() << std::endl;
  
  // Create LAMMPS instance
  LAMMPS *lmp = new LAMMPS(argc, argv, MPI_COMM_WORLD);

  // Run input script (e.g., in.lj)
  lmp->input->file();

  for(int i=0; i<2; ++i) {
    lmp->input->one("run 10 post no");
    
    // Convert atom data to Conduit mesh
    Node mesh;
    lammps_atoms_to_conduit_mesh(lmp, mesh);
    
    // Start Ascent and publish mesh
    Ascent a;
    a.open();
    a.publish(mesh);
    
    // Define actions
    Node actions;
    Node &add_act = actions.append();
    
    std::cout << "Ascent Actions:\n" << actions.to_yaml() << std::endl;
    
    // Render
    a.execute(actions);

    a.close();
  }
  
  // Cleanup
  delete lmp;

  MPI_Finalize(); 
  return 0;
}

In [ ]:
- action: add_scenes
  scenes:
    scene1:
      plots:
        plot1:
          type: pseudocolor
          field: velocity
          points:  
            radius: 0.5  # Increased particle radius
      renders:
        r1:
          image_prefix: "cube_render"
          camera:
            look_at: [0.0, 0.0, 0.0]
            position: [25.0, 25.0, 25.0]
          image_width: 1024
          image_height: 768

In [ ]:
!examples/lammps/cube/build_lammps_ascent_anari.sh

In [ ]:
!examples/lammps/cube/run_ascent_anari.sh &> /dev/null

In [ ]:
!examples/lammps/cube/genmp4.sh

In [ ]:
from IPython.display import Video
Video("lammps_box_anari.mp4")

# Omniverse
Omniverse is a multi-GPU, collaborative simulation and visualization platform built around USD. It provides infrastructure for real-time rendering, physics simulation, and interaction. In this workflow, Omniverse acts as the visualization environment that consumes the USD scene generated by ANARI-USD, allowing immersive inspection and interaction with the evolving simulation state.